In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import re
from copy import deepcopy
import time

In [3]:
years = [x for x in range(2014, 2022)]
leagues = ['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1']

In [4]:
BASE_URL = 'https://understat.com/player/'

In [8]:
url = f"{BASE_URL}6"
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

scripts = soup.find_all("script")

In [6]:
def format_info(shots_string):
	pattern = r"'(.+?)(?=')"
	pattern_capture = re.findall(pattern, shots_string)[0]
	pattern_capture = pattern_capture.encode('utf-8').decode('unicode_escape')
	return json.loads(pattern_capture)

In [13]:
for player_id in tqdm(range(1,4740)):
	shot_list = []
	time.sleep(0.717)
	url = f"{BASE_URL}{player_id}"
	try:
		res = requests.get(url)
		if (res.status_code != 200):
			with open("exceptions.log", "a") as f:
				f.write('player_id: {}, exception: 404\n'.format(
					player_id))
		soup = BeautifulSoup(res.content, "lxml")
		scripts = soup.find_all("script")
		shots_json = ""
		for el in scripts:
			if "shotsData" in el.text:
				full_script_string = el.text.strip()
				shots_info = full_script_string.split("),")[0]
				player_info = full_script_string.split("),")[1]
				shots_json = format_info(shots_info)
				player_json = format_info(player_info)
				df_obj = deepcopy(shots_json)
				for entry in shots_json:
					entry["Player Name"] = player_json["name"]
					entry["Player ID"] = player_json["id"]
					shot_list.append(entry)
		df = pd.DataFrame(shot_list)
		df.to_csv(f"shots/{player_id}.csv")
	except Exception as exception_message:
		with open("exceptions.log", "a") as f:
			f.write('player_id: {}, exception: {}\n'.format(
				player_id, str(exception_message)))


100%|██████████| 4739/4739 [1:47:39<00:00,  1.36s/it]  


In [57]:
df = pd.DataFrame(shot_list)

In [58]:
df.shape

(805715, 22)

In [59]:
df.to_csv("shots/shot_data.csv")

In [54]:
df.shape

(402815, 22)

In [10]:
for el in scripts:
	if "shotsData" in el.text:
		full_script_string = el.text.strip()
		shots_info = full_script_string.split("),")[0]
		player_info = full_script_string.split("),")[1]
		print(format_info(shots_info))


[{'id': '76286', 'minute': '30', 'result': 'Goal', 'X': '0.875', 'Y': '0.6270000076293946', 'xG': '0.16227774322032928', 'player': 'Marcel Heller', 'h_a': 'h', 'player_id': '6', 'situation': 'OpenPlay', 'season': '2015', 'shotType': 'RightFoot', 'match_id': '1026', 'h_team': 'Darmstadt', 'a_team': 'Hannover 96', 'h_goals': '2', 'a_goals': '2', 'date': '2015-08-15 17:30:00', 'player_assisted': 'Marco Sailer', 'lastAction': 'TakeOn'}, {'id': '76292', 'minute': '53', 'result': 'Goal', 'X': '0.9169999694824219', 'Y': '0.3120000076293945', 'xG': '0.5297272801399231', 'player': 'Marcel Heller', 'h_a': 'h', 'player_id': '6', 'situation': 'OpenPlay', 'season': '2015', 'shotType': 'RightFoot', 'match_id': '1026', 'h_team': 'Darmstadt', 'a_team': 'Hannover 96', 'h_goals': '2', 'a_goals': '2', 'date': '2015-08-15 17:30:00', 'player_assisted': None, 'lastAction': 'TakeOn'}, {'id': '76321', 'minute': '47', 'result': 'BlockedShot', 'X': '0.9359999847412109', 'Y': '0.325', 'xG': '0.07438134402036667'